In [1]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

predictions = model(x_train[:1]).numpy()

tf.nn.softmax(predictions).numpy()

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

loss_fn(y_train[:1], predictions).numpy()

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

probability_model(x_test[:5])


11493376/11490434 [==============================] - 1s 0us/step
Epoch 1/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.2920 - accuracy: 0.9141
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1408 - accuracy: 0.9587
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1082 - accuracy: 0.9672
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0888 - accuracy: 0.9723
Epoch 5/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0753 - accuracy: 0.9769
313/313 - 0s - loss: 0.0761 - accuracy: 0.9770


<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.6031600e-09, 1.2489328e-09, 3.4973734e-06, 6.5911910e-04,
        1.2399404e-11, 3.4434160e-07, 4.1058289e-14, 9.9933285e-01,
        6.4453403e-08, 4.1222806e-06],
       [2.7813027e-07, 3.4880628e-05, 9.9894243e-01, 9.9300547e-04,
        2.6209512e-12, 1.7878903e-05, 1.1471030e-08, 7.0002795e-11,
        1.1604306e-05, 3.1244807e-10],
       [1.7255886e-07, 9.9949408e-01, 1.2639433e-05, 4.3210412e-06,
        2.6506441e-05, 2.4879050e-06, 2.7735509e-06, 4.4066360e-04,
        1.6059530e-05, 3.0514806e-07],
       [9.9905115e-01, 2.2971147e-08, 4.0349705e-04, 6.0095026e-06,
        1.1576022e-05, 1.7452288e-05, 3.2114412e-04, 1.2902556e-04,
        6.0836355e-06, 5.3941192e-05],
       [3.0667297e-06, 5.5017475e-09, 1.7781568e-06, 5.0721758e-07,
        9.9004000e-01, 1.2110104e-05, 7.7145273e-07, 3.7910524e-04,
        1.9967472e-06, 9.5606148e-03]], dtype=float32)>